# Analisis Heuristico de Equipos

## Librerias y variables

In [1]:
# === 📚 Importacion de librerías ===
import csv
import requests
import os
import json
import re
import pandas as pd
from collections import defaultdict
from datetime import datetime, timedelta, date
from google.colab import userdata

In [2]:
# === ⚙️ Variables generales ===
archivo_Series = "series_frappe.csv"
archivo_crudo = "crudo.csv"
archivo_seriesFiltrados = "series_filtrados.csv"

# URL directa del CSV (sin filtro)
url_crudo_docs = userdata.get('planilla_anual')




enviar=1


# === 🔐 API ERP ===
API_KEY = userdata.get('API_KEY')
API_SECRET = userdata.get('API_SECRET')
try:
  FRAPPE_API_crudo = requests.get("https://docs.google.com/spreadsheets/d/e/2PACX-1vRbnN152KxnzZqA3GA0L0LQJCF9uGRNIXsydi_FB1EveJ_aVSD_5rjHtTcfoGyctN8ZPou5R7Eg-QQ8/pub?gid=1381182595&single=true&output=csv").text.splitlines()[1]
  FRAPPE_API=FRAPPE_API_crudo + "/api/resource"
except requests.exceptions.RequestException as e:
    print(f"❌ Error fatal: No se pudo obtener la URL de la API de Frappe desde Google Docs. Revisa el enlace. Error: {e}")
    FRAPPE_API = None # Dejar la API como None para detener la ejecución si falla.

## Funciones menores

In [3]:
def agregar_comentario(doctype: str, docname: str, contenido: str, remitente: str = "aguida@qpsrl.com.ar"):
    if not FRAPPE_API_crudo:
        print("❌ No se puede agregar comentario, la URL de la API de Frappe no está configurada.")
        return

    try:
        url = f"{FRAPPE_API_crudo}/api/method/frappe.desk.form.utils.add_comment"
        payload = {
            "reference_doctype": doctype, "reference_name": docname, "content": contenido,
            "comment_email": remitente, "comment_by": remitente
        }
        headers = {"Authorization": f"token {API_KEY}:{API_SECRET}", "Content-Type": "application/json"}
        response = requests.post(url, headers=headers, json=payload)
        response.raise_for_status()
        print(f"🗨️ Comentario agregado exitosamente a {docname}.")
    except requests.exceptions.RequestException as e:
        error_text = e.response.text if e.response else "Sin respuesta del servidor"
        print(f"❌ Error al enviar comentario a Frappe para {docname}: {error_text}")
    except Exception as e:
        print(f"❌ Error inesperado en agregar_comentario: {e}")

def crear_delivery_note_por_serie(serie_detectada, fila_crudo, enviar=1):
    if not FRAPPE_API: return None

    dic_series = {row["name"].strip(): {"item_code": row["item_code"].strip(),
                                       "item_name": row["item_name"].strip(),
                                       "warehouse": row["warehouse"].strip()}
                  for row in csv.DictReader(open(archivo_Series, newline='', encoding="utf-8"))}

    info_serie = dic_series.get(serie_detectada)

    # Verificación adicional (aunque ya debería estar filtrado)
    if not info_serie or not info_serie["warehouse"]:
        print(f"⚠️ No se puede crear DN para '{serie_detectada}'. No se encontró o no tiene almacén.")
        return None

    payload = {
        "customer": "Instalado", "posting_date": str(date.today()), "set_warehouse": info_serie["warehouse"],
        "docstatus": enviar, "items": [{"item_code": info_serie["item_code"], "item_name": info_serie["item_name"],
                                  "qty": 1, "warehouse": info_serie["warehouse"], "serial_no": serie_detectada}],
    }

    FRAPPE_API_delivery = f"{FRAPPE_API}/Delivery%20Note"
    HEADERS = {
        "Authorization": f"token {API_KEY}:{API_SECRET}",
        "Content-Type": "application/json",
        "ngrok-skip-browser-warning": "true"
    }

    print(f"📦 Creando Delivery Note para la serie '{serie_detectada}'...")
    try:
        # Crear sesión con retries
        session = requests.Session()
        session.headers.update(HEADERS)

        response = session.post(FRAPPE_API_delivery, json=payload)
        response.raise_for_status()
        docname = response.json()["data"]["name"]
        print(f"✅ Delivery Note {FRAPPE_API_crudo}/app/delivery-note/{docname} creada exitosamente.")

        headers = "".join(f"<th style='padding: 5px; border: 1px solid #ccc;'>{col}</th>" for col in fila_crudo.keys())
        values  = "".join(f"<td style='padding: 5px; border: 1px solid #ccc;'>{val}</td>" for val in fila_crudo.values())

        comentario_tabla = (
            "<b>Detalle de la fila encontrada en el archivo crudo de instalaciones:</b><br>"
            "<table border='1' style='width:100%; border-collapse: collapse; margin-top: 10px;'>"
            f"<tr>{headers}</tr>"
            f"<tr>{values}</tr>"
            "</table>"
        )

        agregar_comentario(doctype="Delivery Note", docname=docname, contenido=comentario_tabla)

        comentario_automatico = "<p style='margin-top:15px;'>Este documento fue generado automáticamente por el script de trazabilidad debido a una <b>coincidencia exacta</b> del número de serie.</p>"
        agregar_comentario(doctype="Delivery Note", docname=docname, contenido=comentario_automatico)

        return docname
    except requests.exceptions.RequestException as e:
        print(f"❌ Error creando Delivery Note para '{serie_detectada}': {e.response.text if e.response else e}")
        return None

## Funciones de obtencion de datos

In [4]:
# === 🛠️ Funciones de Descarga y Preparación ===

def descargar_crudo_completo():
    print("Descargando archivo crudo de instalaciones...")
    try:
        resp = requests.get(url_crudo_docs)
        resp.raise_for_status()
        with open(archivo_crudo, "w", newline='', encoding="utf-8") as f:
            f.write(resp.text)
        print(f"✅ Crudo completo guardado en '{archivo_crudo}'")
        return True
    except requests.exceptions.RequestException as e:
        print(f"❌ Error al descargar el archivo crudo: {e}")
        return False

def obtener_series_frappe(lista_nombres=None):
    if not FRAPPE_API:
        return False

    campos = [
        "name", "docstatus", "item_code", "item_name",
        "warehouse", "creation", "modified", "_user_tags"
    ]

    # Si se proporciona una lista de nombres, crear registros para cada uno
    if lista_nombres and isinstance(lista_nombres, list) and len(lista_nombres) > 0:
        print(f"Creando archivo con {len(lista_nombres)} series de la lista...")
        data = []
        for nombre in lista_nombres:
            registro = {campo: "" for campo in campos}  # Crear registro vacío
            registro["name"] = nombre  # Llenar el campo name
            registro["warehouse"] = "1"  # Poner "1" en warehouse para series de la lista
            data.append(registro)
    else:
        # Comportamiento normal: obtener todos los series de Frappe
        print("Consultando todas las series en Frappe...")
        FRAPPE_API_serie = FRAPPE_API + "/Serial%20No"
        HEADERS = {
            "Authorization": f"token {API_KEY}:{API_SECRET}"
        }

        order_by_param = "modified asc"
        fields_param = json.dumps(campos)
        url = f"{FRAPPE_API_serie}?fields={fields_param}&limit_page_length=30000&order_by={order_by_param}"

        try:
            response = requests.get(url, headers=HEADERS)
            response.raise_for_status()
            data = response.json().get("data", [])
        except (requests.exceptions.RequestException, json.JSONDecodeError) as e:
            print(f"❌ Error obteniendo series de Frappe: {e}")
            return False

    # Escribir el archivo CSV
    try:
        with open(archivo_Series, "w", newline='', encoding="utf-8") as f:
            writer = csv.DictWriter(f, fieldnames=campos)
            writer.writeheader()
            writer.writerows(data)

        print(f"✅ {len(data)} series guardadas en '{archivo_Series}'")
        return True

    except Exception as e:
        print(f"❌ Error escribiendo archivo CSV: {e}")
        return False

# === 🧠 Funciones de Trazabilidad y Análisis ===

def obtener_series_filtradas(filtro_tiempo=False):
    series_a_buscar = []
    datos_para_csv = []
    dos_semanas_atras = datetime.now() - timedelta(weeks=2)

    with open(archivo_Series, newline='', encoding="utf-8") as f:
        reader = csv.DictReader(f)
        for fila in reader:
            warehouse = fila.get("warehouse", "").strip()
            serie = fila["name"].strip().upper()

            # 🔴 Siempre descartar si no hay warehouse
            if not warehouse:
                continue

            cumple_filtro_tiempo = True
            if filtro_tiempo:
                try:
                    # New, correct date format: "%Y-%m-%d %H:%M:%S.%f"
                    fecha_mod = datetime.strptime(fila["modified"], "%Y-%m-%d %H:%M:%S.%f")
                    if fecha_mod >= dos_semanas_atras:
                        cumple_filtro_tiempo = False
                except (ValueError, TypeError):
                    # Handle cases where the date format is different or the field is empty
                    # For safety, we'll assume it doesn't meet the time filter if we can't parse it.
                    cumple_filtro_tiempo = False
                    print(f"⚠️ Advertencia: No se pudo parsear la fecha '{fila.get('modified', 'N/A')}' para la serie {serie}. No se incluirá en el filtro de tiempo.")

            if cumple_filtro_tiempo:
                series_a_buscar.append(serie)
                datos_para_csv.append({
                    "serie": serie, "almacen": warehouse, "item_code": fila["item_code"],
                    "item_name": fila["item_name"], "modified": fila["modified"]
                })

    with open(archivo_seriesFiltrados, "w", newline='', encoding="utf-8") as f_out:
        writer = csv.DictWriter(f_out, fieldnames=["serie", "almacen", "item_code", "item_name", "modified"])
        writer.writeheader()
        writer.writerows(datos_para_csv)

    print(f"🎯 Se prepararon {len(series_a_buscar)} series para la trazabilidad (todas con almacén no vacío).")
    return series_a_buscar


## Funciones de busqueda

In [5]:
def buscar_series_en_crudo(series_buscar):
    encontrados = {}
    series_a_buscar_set = set(series_buscar)

    with open(archivo_crudo, newline='', encoding="utf-8") as f:
        reader = csv.DictReader(f)  # Usar DictReader en lugar de reader
        for fila in reader:
            # Obtener los valores de las columnas por nombre en lugar de índice
            columna1 = fila.get('SERIAL ONT', '')
            columna9 = fila.get('SERIALES DECO FLOW', '')
            columna10 = fila.get('COMENTARIOS DE CIERRE', '')

            texto_crudo = " ".join([columna1, columna9, columna10]).upper()

            for serie in series_a_buscar_set:
                if serie in texto_crudo and serie not in encontrados:
                    encontrados[serie] = fila

    print(f"✔️ Se encontraron {len(encontrados)} coincidencias exactas en el archivo crudo.")
    return encontrados

def distancia_levenshtein(a, b):
    if len(a) < len(b): return distancia_levenshtein(b, a)
    if len(b) == 0: return len(a)
    previous_row = range(len(b) + 1)
    for i, c1 in enumerate(a):
        current_row = [i + 1]
        for j, c2 in enumerate(b):
            insertions = previous_row[j + 1] + 1
            deletions = current_row[j] + 1
            substitutions = previous_row[j] + (c1 != c2)
            current_row.append(min(insertions, deletions, substitutions))
        previous_row = current_row
    return previous_row[-1]

def buscar_series_en_crudo_heuristico(series_buscar, max_distancia=3):
    sugerencias = []
    series_frappe_info = {}

    # Leer todas las series de Frappe para verificar existencia y almacén
    with open(archivo_Series, newline='', encoding='utf-8') as f:
        reader = csv.DictReader(f)
        for row in reader:
            series_frappe_info[row['name'].strip().upper()] = row['warehouse'].strip()

    with open(archivo_crudo, newline='', encoding="utf-8") as f:
        reader = csv.reader(f)
        next(reader, None)
        for i, fila in enumerate(reader):
            if len(fila) < 11: continue
            texto_crudo = " ".join([fila[1], fila[9], fila[10]]).upper()
            imagenes = fila[8].strip().split(",") if len(fila) > 8 else []
            palabras = set(re.findall(r"[A-Z0-9]{8,}", texto_crudo))

            for serie_objetivo in series_buscar:
                for palabra in palabras:
                    if serie_objetivo == palabra:
                        continue

                    dist = distancia_levenshtein(serie_objetivo, palabra)
                    if 0 < dist <= max_distancia:
                        # VALIDACIÓN MODIFICADA según los nuevos requisitos
                        warehouse_de_palabra = series_frappe_info.get(palabra)

                        # Solo considerar si:
                        # 1. La palabra NO existe en la base de datos (warehouse_de_palabra es None)
                        # O 2. Si existe, debe tener un warehouse no vacío
                        if warehouse_de_palabra is not None and not warehouse_de_palabra:
                            continue  # Existe pero tiene warehouse vacío, descartar

                        confianza = round(1 - dist / max(len(serie_objetivo), len(palabra)), 2)
                        sugerencias.append({
                            "serie_buscada": serie_objetivo,
                            "warehouse_buscada": series_frappe_info.get(serie_objetivo, "N/A"),
                            "posible_coincidencia": palabra,
                            "distancia": dist,
                            "confianza_percent": int(confianza * 100),
                            "fila_crudo_num": i + 2,
                            "imagenes": [img.strip() for img in imagenes if img.strip()]
                        })

    sugerencias.sort(key=lambda x: x["confianza_percent"], reverse=True)
    print(f"\n🤖 Se generaron {len(sugerencias)} sugerencias heurísticas (distancia <= {max_distancia}):\n")

    for s in sugerencias:
        print(f" - Serie Buscada: {s['serie_buscada']} (Almacén: {s['warehouse_buscada']})")
        print(f"   Posible Coincidencia: {s['posible_coincidencia']}")
        print(f"   Posibilidad de match: {s['confianza_percent']}% (Distancia: {s['distancia']})")
        print(f"   Ubicación en Crudo: Fila {s['fila_crudo_num']} | Imágenes: {s['imagenes']}\n")

    return sugerencias


## Main

In [6]:
# === 🚀 EJECUCIÓN PRINCIPAL DEL PROCESO DE TRAZABILIDAD ===

def main(filtro_tiempo=False, send=1):
    max_distancia_heuristica = 2 if filtro_tiempo else 2
    modo_desc = "series sin movimiento en 2 semanas" if filtro_tiempo else "todas las series con stock"
    print(f"--- INICIANDO ANÁLISIS DE TRAZABILIDAD ---")
    print(f"Modo de operación: Buscando {modo_desc}.")
    print(f"Distancia heurística máxima establecida en: {max_distancia_heuristica}")

    #series=["ALCLB26BE330", "48575443885A3FB0"]
    series=[]
    if not descargar_crudo_completo() or not obtener_series_frappe(series):
        print("\n🛑 Proceso detenido por error en la descarga de datos iniciales.")
        return

    print("\n--- ETAPA 1: FILTRANDO SERIES A BUSCAR ---")
    series_para_buscar = obtener_series_filtradas(filtro_tiempo=filtro_tiempo)
    if not series_para_buscar:
        print("\nNo se encontraron series que cumplan con los criterios de filtro (con almacén no vacío). El proceso ha finalizado.")
        return

    print("\n--- ETAPA 2: BUSCANDO COINCIDENCIAS EXACTAS Y CREANDO DELIVERY NOTES ---")

    # Obtener la fecha de hoy para la comparación
    fecha_hoy = datetime.now().date()

    coincidencias_exactas = buscar_series_en_crudo(series_para_buscar)

    for serie, fila_crudo in coincidencias_exactas.items():
        print(f"\nProcesando coincidencia exacta para: {serie}")
        # Access the date column by its name instead of index
        # Based on the previous code, the date is likely in the second column.
        # You might need to adjust the key name if the column name is different in your CSV.
        fecha_registro_str = fila_crudo.get('FECHA DE CIERRE') # Replace 'FECHA DE REGISTRO' with the actual column name for the date

        if fecha_registro_str:
            try:
                fecha_registro = datetime.strptime(fecha_registro_str, '%d/%m/%Y').date()

                if fecha_registro == fecha_hoy:
                    print(f"✅ ¡Atención! Se encontró una posible Delivery Note para la serie {serie}, pero la fecha de registro ({fecha_registro}) es del día de hoy. No se creará la Delivery Note.")
                else:
                    crear_delivery_note_por_serie(serie, fila_crudo, send)
            except ValueError:
                print(f"⚠️ Error: No se pudo parsear la fecha '{fecha_registro_str}' para la serie {serie}. Se intentará crear la Delivery Note de todos modos.")
                crear_delivery_note_por_serie(serie, fila_crudo, send)
        else:
            print(f"⚠️ Advertencia: No se encontró la columna de fecha para la serie {serie}. Se intentará crear la Delivery Note de todos modos.")
            crear_delivery_note_por_serie(serie, fila_crudo, send)


    # --- FIN DE MODIFICACIÓN ---

    print("\n--- ETAPA 3: BUSCANDO SUGERENCIAS HEURÍSTICAS (ERRORES DE TIPEO) ---")
    series_encontradas_exactas = set(coincidencias_exactas.keys())
    series_no_encontradas = [s for s in series_para_buscar if s not in series_encontradas_exactas]

    if series_no_encontradas:
        print(f"{len(series_no_encontradas)} series no tuvieron coincidencia exacta. Iniciando búsqueda heurística...")
        sugerencias_heuristicas = buscar_series_en_crudo_heuristico(series_no_encontradas, max_distancia=max_distancia_heuristica)

        series_encontradas_heuristica = set(s['serie_buscada'] for s in sugerencias_heuristicas)

        # Find series that were not found in either search
        if filtro_tiempo==True:
          series_sin_match = [s for s in series_no_encontradas if s not in series_encontradas_heuristica]
          # Print the result

          if series_sin_match:
              print(f"\n🚨 Se encontraron {len(series_sin_match)} series que pasaron el filtro de tiempo, pero no se encontraron en ninguna de las búsquedas:")

              # Abrir el archivo filtrado para buscar almacenes
              with open(archivo_seriesFiltrados, newline='', encoding="utf-8") as f:
                  reader = csv.DictReader(f)
                  almacen_por_serie = {fila["serie"]: fila["almacen"] for fila in reader}

              # Mostrar cada serie con su almacén
              for s in series_sin_match:
                  almacen = almacen_por_serie.get(s, "N/A")
                  print(f" - {s:<16} - {almacen}")


          else:
              print("\n🎉 Todas las series que pasaron el filtro de tiempo fueron encontradas (ya sea con coincidencia exacta o heurística).")

    else:
        print("Todas las series filtradas fueron encontradas con coincidencia exacta. No es necesaria la búsqueda heurística.")

    print("\n✅ Proceso de trazabilidad completado.")

if __name__ == "__main__":
    print("\n" + "="*56)
    print("|| EJECUTANDO ANÁLISIS DE SERIES CON ALMACÉN NO VACÍO ||")
    print("="*56 + "\n")
    main(filtro_tiempo=False, send=enviar)
    main(filtro_tiempo=True, send=enviar)


|| EJECUTANDO ANÁLISIS DE SERIES CON ALMACÉN NO VACÍO ||

--- INICIANDO ANÁLISIS DE TRAZABILIDAD ---
Modo de operación: Buscando todas las series con stock.
Distancia heurística máxima establecida en: 2
Descargando archivo crudo de instalaciones...
✅ Crudo completo guardado en 'crudo.csv'
Consultando todas las series en Frappe...
✅ 2193 series guardadas en 'series_frappe.csv'

--- ETAPA 1: FILTRANDO SERIES A BUSCAR ---
🎯 Se prepararon 155 series para la trazabilidad (todas con almacén no vacío).

--- ETAPA 2: BUSCANDO COINCIDENCIAS EXACTAS Y CREANDO DELIVERY NOTES ---
✔️ Se encontraron 2 coincidencias exactas en el archivo crudo.

Procesando coincidencia exacta para: FZ324060356542
✅ ¡Atención! Se encontró una posible Delivery Note para la serie FZ324060356542, pero la fecha de registro (2025-10-20) es del día de hoy. No se creará la Delivery Note.

Procesando coincidencia exacta para: FZ324060367339
✅ ¡Atención! Se encontró una posible Delivery Note para la serie FZ324060367339, pero